In [ ]:
http://scipy.github.io/devdocs/stats.mstats.html
https://github.com/GRousselet/blog/tree/master/harrell-davis
https://rdrr.io/github/GRousselet/rogme/src/R/stats.R

In [7]:
import numpy as np 
import random 
import math as m 
import scipy.special 


n = 10000       #total number of simulations

A1 = np.zeros(n)   #Asset 1 with n returns 
A2 = np.zeros(n)
A3 = np.zeros(n)
Return = np.zeros(n)    #Overall Return of the Portfolio
weights = [0.2,0.45,0.35]    #Weights of each asset in the portfolio

if sum(weights) != 1: 
    raise ValueError('Sum of the weights should equal 1')
    
for i in range(n): 
    epsilon = random.gauss(0,0.5)   #to do correlation between the asset classes
    X_i = random.gauss(0,1)     #picking random float number where mean=0 and variance = 1
    A1[i] = X_i
    A2[i] = random.uniform(X_i,X_i+epsilon)   #Correlating A2 such that it is always greater than A1
    A3[i] = random.uniform(A2[i]-epsilon,A2[i])
    Return[i] = weights[0]*A1[i] + weights[1]*A2[i] + weights[2]*A3[i]      #Calculating return of portfolio

order_Return = sorted(Return,key = float)       #Sorting it in ascending order so we can pick the 5% th for VaR

Order_A1 = np.array(sorted(A1,key = float))      #Also placing Assets in ascending order
Order_A2 = np.array(sorted(A2,key = float))
Order_A3 = np.array(sorted(A3,key = float))

p = 0.05   #quantile

def W(A, p):
#Calculates the Harrell Davis Weights for a given Asset Class '''
        n = len(A)
        return [(scipy.special.betainc(p*(n + 1), (1 - p)*(n + 1), (i + 1)/float(n)))- 
                (scipy.special.betainc(p*(n + 1), (1 - p)*(n + 1), (i)/float(n))) for i in range(n)]

print('Individual VaR for A1: %f' %(W(A1,p)@Order_A1))   #Calculates individual VaR by doing dot product of HD Weights @ Order_A1
print('Individual VaR for A2: %f' %(W(A2,p)@Order_A2))
print('Individual VaR for A3: %f' %(W(A3,p)@Order_A3))

#Summing for Portfolio VaR
Sum_Component_VaR = weights[0]*(W(A1,p)@Order_A1) + weights[1]*(W(A2,p)@Order_A2) + weights[2]*(W(A3,p)@Order_A3)  

print()

print('Sum of component VaR: %f' %(Sum_Component_VaR))
print('Total Standalone VaR: %f' %order_Return[m.floor(n*p-1)])  #Checking that Sum_Component_VaR approx. Standalone VaR

delta = 0.01
Relative_error = abs(Sum_Component_VaR-order_Return[m.floor(n*p-1)])/abs(order_Return[m.floor(n*p-1)])

print()

if  Relative_error <= delta : 
    print('Harrell Davis gives a good approximation of VaR of portfolio with Statistic Error of %0.2f%%' %(Relative_error*100))
else: 
    print('Harrell Davis gives poor approximation of Standalone VaR with error of %0.2f%%' %(Relative_error*100))

Individual VaR for A1: -1.620757
Individual VaR for A2: -1.710293
Individual VaR for A3: -1.654191

Sum of component VaR: -1.672750
Total Standalone VaR: -1.655662

Harrell Davis gives poor approximation of Standalone VaR with error of 1.03%
